# 作业02：Fashion Mnist实现图像分类

## 1 概述
Fashion-MNIST是一个替代MNIST手写数字集的图像数据集，它是由Zalando（一家德国的时尚科技公司）旗下的研究部门提供。其涵盖了来自10种类别的共7万个不同商品的正面图片。Fashion-MNIST的大小、格式和训练集/测试集划分与原始的MNIST完全一致。数据集按照60000/10000的比例进行训练测试数据划分，全部数据均为28x28的灰度图片。

**基本要求**：
1. 使用卷积神经网络完成Fashion-MNIST的分类任务。
2. 在README文件中描述所使用的模型的结构、优化器、损失函数和超参数等信息，以及模型在训练集和测试集上的最优结果。
3. 对模型中训练过程损失函数的变化趋势可视化

**加分项**：可以对模型进行优化（包括增加层数，使用残差连接，调整超参数等），或者对模型的训练过程/结果进行可视化（例如模型损失函数在训练过程中的变化趋势，或者参数的分布随训练批次的变化趋势等）。

## 2. 实现模型

### 2.1 数据集加载
可以通过飞桨自带的`paddle.vision.dataset`进行数据集加载，并且通过`paddle.vision.transforms`对数据进行预处理，例如对数据进行归一化。

In [1]:
import paddle
print(paddle.__version__)

import paddle.vision.transforms as T

transform = T.Compose([T.Normalize(mean=[127.5],
                                   std=[127.5],
                                   data_format='CHW')])

# 对数据进行归一化
train_dataset = paddle.vision.datasets.FashionMNIST(mode='train', transform=transform)
test_dataset = paddle.vision.datasets.FashionMNIST(mode='test', transform=transform)

2.1.2


Cache file /home/aistudio/.cache/paddle/dataset/fashion-mnist/train-images-idx3-ubyte.gz not found, downloading https://dataset.bj.bcebos.com/fashion_mnist/train-images-idx3-ubyte.gz 
Begin to download

Download finished
Cache file /home/aistudio/.cache/paddle/dataset/fashion-mnist/train-labels-idx1-ubyte.gz not found, downloading https://dataset.bj.bcebos.com/fashion_mnist/train-labels-idx1-ubyte.gz 
Begin to download
........
Download finished
Cache file /home/aistudio/.cache/paddle/dataset/fashion-mnist/t10k-images-idx3-ubyte.gz not found, downloading https://dataset.bj.bcebos.com/fashion_mnist/t10k-images-idx3-ubyte.gz 
Begin to download

Download finished
Cache file /home/aistudio/.cache/paddle/dataset/fashion-mnist/t10k-labels-idx1-ubyte.gz not found, downloading https://dataset.bj.bcebos.com/fashion_mnist/t10k-labels-idx1-ubyte.gz 
Begin to download
..
Download finished


### 2.2 模型搭建

使用paddle的接口搭建神经网络模型。以LeNet模型为例，该模型与1998年提出，包含了深度学习图像处理相关的基本模块。

LeNet模型一共有7层，包括2个**卷积层**，2个**池化层**和3个**全连接层**。通过连续使用卷积层和池化层提取图像特征。Paddle中提供了相应的接口，可以快速搭建网络模型：
1. 卷积层：`paddle.nn.Conv2D`；
2. 池化层：`paddle.nn.MaxPool2D`；
3. 全连接层：`paddle.nn.Linear`。

### **作业要求**：搭建神经网络，完成`__init__`和`forward`函数，并在README文件中详细描述所使用模型的信息。

In [2]:
import paddle
import paddle.nn as nn
import paddle.nn.functional as F

# 首先实现中间两个卷积层，Skip Connection 1x1 卷积层的残差模块。代码如下：
# 残差模块
class Residual(nn.Layer):
    def __init__(self, in_channel, out_channel, use_conv1x1=False, stride=1):
        super(Residual, self).__init__()
        
        # 第一个卷积单元
        self.conv1 = nn.Conv2D(in_channel, out_channel, kernel_size=3, padding=1, stride=stride)
        self.bn1 = nn.BatchNorm2D(out_channel)
        self.relu = nn.ReLU()

        # 第二个卷积单元
        self.conv2 = nn.Conv2D(out_channel, out_channel, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2D(out_channel)

        if use_conv1x1: #使用1x1卷积核完成shape匹配,stride=2实现下采样
            self.skip = nn.Conv2D(in_channel, out_channel, kernel_size=1, stride=stride)
        else:
            self.skip = None
        

    def forward(self, x):
        # 前向计算
        # [b, c, h, w], 通过第一个卷积单元
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        # 通过第二个卷积单元
        out = self.conv2(out)
        out = self.bn2(out)
        #  通过 identity 模块
        if self.skip:
            x = self.skip(x)
        #  2 条路径输出直接相加,然后输入激活函数
        output = F.relu(out + x)

        return output


# 通过build_resblock 可以一次完成2个残差模块的创建。代码如下：
def build_resblock(in_channel, out_channel, num_layers, is_first=False):
    if is_first:
        assert in_channel == out_channel
    block_list = []
    for i in range(num_layers):
        if i == 0 and not is_first:
            block_list.append(Residual(in_channel, out_channel, use_conv1x1=True, stride=2))
        else:
            block_list.append(Residual(out_channel, out_channel))
    resNetBlock = nn.Sequential(*block_list) #用*号可以把list列表展开为元素
    return resNetBlock

# 下面来实现ResNet18网络模型。代码如下：
class ResNet18_1(nn.Layer):
    # 继承paddle.nn.Layer定义网络结构
    def __init__(self,num_classes=10):
        super(ResNet18_1, self).__init__()
        # 初始化函数(根网络，预处理)
        # x:[b, c, h ,w]=[b,3,224,224]
        self.features = nn.Sequential(
            nn.Conv2D(in_channels=1, out_channels=64, kernel_size=7, 
            stride=2, padding=3),# 第一层卷积,x:[b,64,112,112]
            nn.BatchNorm2D(64),# 归一化层
            nn.ReLU(),
            nn.MaxPool2D(kernel_size=3, stride=2, padding=1)# 最大池化，下采样,x:[b,64,56,56]
        )
        
        # 堆叠 4 个 Block，每个 block 包含了多个残差模块,设置步长不一样
        self.layer1 = build_resblock(64, 64, 2, is_first=True) # x:[b,64,56,56]
        self.layer2 = build_resblock(64, 150, 2) # x:[b,150,28,28]
        self.layer3 = build_resblock(150, 360, 2)  # x:[b,360,14,14]
        self.layer4 = build_resblock(360, 720, 2)  # x:[b,720,7,7]

        # 通过 Pooling 层将高宽降低为 1x1,[b,720,1,1]
        self.avgpool = nn.AdaptiveAvgPool2D(1)
        # 需要拉平为[b,720],不能直接输出连接线性层
        self.flatten = nn.Flatten()
        # 最后连接一个全连接层分类
        self.fc = nn.Linear(in_features=720,out_features=num_classes)

    def forward(self, inputs):
        # 前向计算函数：通过根网络
        x = self.features(inputs)
        # 一次通过 4 个模块
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        # 通过池化层
        x = self.avgpool(x)
        # 拉平
        x = self.flatten(x)
        # 通过全连接层
        x = self.fc(x)
        return x

# ResNet18网络
model = ResNet18_1()
# 可视化模型
paddle.summary(model,(-1,1,28,28))

-------------------------------------------------------------------------------
   Layer (type)         Input Shape          Output Shape         Param #    
     Conv2D-1         [[1, 1, 28, 28]]     [1, 64, 14, 14]         3,200     
   BatchNorm2D-1     [[1, 64, 14, 14]]     [1, 64, 14, 14]          256      
      ReLU-1         [[1, 64, 14, 14]]     [1, 64, 14, 14]           0       
    MaxPool2D-1      [[1, 64, 14, 14]]      [1, 64, 7, 7]            0       
     Conv2D-2         [[1, 64, 7, 7]]       [1, 64, 7, 7]         36,928     
   BatchNorm2D-2      [[1, 64, 7, 7]]       [1, 64, 7, 7]           256      
      ReLU-2          [[1, 64, 7, 7]]       [1, 64, 7, 7]            0       
     Conv2D-3         [[1, 64, 7, 7]]       [1, 64, 7, 7]         36,928     
   BatchNorm2D-3      [[1, 64, 7, 7]]       [1, 64, 7, 7]           256      
    Residual-1        [[1, 64, 7, 7]]       [1, 64, 7, 7]            0       
     Conv2D-4         [[1, 64, 7, 7]]       [1, 64, 7, 7]     

{'total_params': 21516732, 'trainable_params': 21495772}

## 3. 模型配置与训练
可以使用`Model`搭建实例，然后使用`model.prepare`接口进行模型的配置，比如优化器、损失函数和评价指标等，也可以使用其他方法配置和训练模型。

In [3]:
model = paddle.Model(model) # 用Model封装模型

# 配置模型
model.prepare(paddle.optimizer.Adam(learning_rate=0.001, parameters=model.parameters()),
              paddle.nn.CrossEntropyLoss(),
              paddle.metric.Accuracy())

### **作业要求**：完成训练过程可视化的相关函数，训练模型并保存可视化结果。

In [5]:
import matplotlib.pyplot as plt
# 参数的分布图， 损失函数， acc

def DrawLossFunction():
    pass

# 训练可视化VisualDL工具的回调函数
visualdl = paddle.callbacks.VisualDL(log_dir='visualdl_log')

model.fit(train_dataset, test_dataset, epochs=50, batch_size=64, shuffle=True, verbose=1,save_dir='./chk_points/', # 分阶段的训练模型存储路径
          save_freq=5,             # 保存模型的频率 
          callbacks=[visualdl])

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/50


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:77: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return (isinstance(seq, collections.Sequence) and
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/nn/layer/norm.py:641: UserWarning: When training, we now always track global mean and variance.
  "When training, we now always track global mean and variance.")


step 938/938 [==============================] - loss: 0.3626 - acc: 0.8386 - 24ms/step        
save checkpoint at /home/aistudio/chk_points/0
Eval begin...
step 157/157 [==============================] - loss: 0.3061 - acc: 0.8723 - 14ms/step        
Eval samples: 10000
Epoch 2/50
step 938/938 [==============================] - loss: 0.2129 - acc: 0.8893 - 25ms/step         
Eval begin...
step 157/157 [==============================] - loss: 0.1945 - acc: 0.8856 - 14ms/step        
Eval samples: 10000
Epoch 3/50
step 938/938 [==============================] - loss: 0.4254 - acc: 0.8994 - 25ms/step         
Eval begin...
step 157/157 [==============================] - loss: 0.3567 - acc: 0.8861 - 14ms/step        
Eval samples: 10000
Epoch 4/50
step 938/938 [==============================] - loss: 0.4004 - acc: 0.9097 - 27ms/step        
Eval begin...
step 157/157 [==============================] - loss: 0.2714 - acc: 0.8876 - 14ms/step        
Eval samples: 10000
Epoch 5/50
step 938/93

In [11]:
model.evaluate(test_dataset, verbose=1)

Eval begin...
step 10000/10000 [==============================] - loss: 6.0338 - acc: 0.9083 - 10ms/step        
Eval samples: 10000


{'loss': [6.033785], 'acc': 0.9083}